In [1]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import gradio as gr

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

import sys
sys.path.append("D:/Educational/TensorFlow/models/research/")
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

In [2]:
# Activate GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#     # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#         print(e)

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path (this can be local or on colossus)

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [4]:
category_index = {
    1: {'id': 1, 'name': 'apple'},
    2: {'id': 2, 'name': 'banana'},
    3: {'id': 3, 'name': 'orange'},
}

In [5]:
start_time = time.time()
tf.keras.backend.clear_session()
# detect_fn = tf.saved_model.load('exported-models/model1/saved_model/')
detect_fn = tf.saved_model.load('exported-models/model1/saved_model/')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

Elapsed time: 30.688602924346924s


In [6]:
def predict(image_path):
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    plt.rcParams['figure.figsize'] = [42, 21]
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)
    return image_np_with_detections
image_path = "D:/Educational/TensorFlow/workspace/training_demo/images/test/mixed_3.jpg"
image_np_with_detections = predict(image_path)
plt.axis('off')
plt.imshow(image_np_with_detections)

In [7]:
# Turn off gpu, comment out image_np = load_image_into_numpy_array(image_path) as gradio is already giving np array
# iface = gr.Interface(predict, gr.inputs.Image(), "image")
# iface.launch(share=True)

Running locally at: http://127.0.0.1:7860/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://46809.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://46809.gradio.app')

In [6]:
model = tf.saved_model.load('exported-models/model1/saved_model/')

In [7]:
model.save('exported-models/Model1.h5')

AttributeError: '_UserObject' object has no attribute 'save'